# iotSDR GNSS Recorder Notebook:


## Setting up the environment:

* import required python modules



In [1]:
from pynq import Overlay
import iotSDR.iotSDR_GPS_streamer as iotSDR_GPS_streamer
import iotSDR.max2769_driver as max2769_driver
import time
import os

In [2]:
recording_file = "./raw_smples_10.bin"
recording_time = 3 #time for recording in seconds
wordsPerBlk    = 4*1024

In [3]:
overlay = Overlay('../bitstream/iotSDR_GPS_Receiver.bit', download=True)
if not overlay.is_loaded():
    overlay.download()
    print("Bitfile now Loaded")
    
overlay.ip_dict


{'GPS_Receiver_IQ_Streamer/axi_dma_0': {'fullpath': 'GPS_Receiver_IQ_Streamer/axi_dma_0',
  'type': 'xilinx.com:ip:axi_dma:7.1',
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1077936128,
  'mem_id': 'S_AXI_LITE',
  'gpio': {},
  'interrupts': {'s2mm_introut': {'controller': 'axi_intc_0',
    'index': 0,
    'fullpath': 'GPS_Receiver_IQ_Streamer/axi_dma_0/s2mm_introut'}},
  'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_DLYTMR_RESOLUTION': '125',
   'C_PRMRY_IS_ACLK_ASYNC': '0',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_INCLUDE_SG': '0',
   'C_SG_INCLUDE_STSCNTRL_STRM': '0',
   'C_SG_USE_STSAPP_LENGTH': '0',
   'C_SG_LENGTH_WIDTH': '26',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32',
   'C_MICRO_DMA': '0',
   'C_INCLUDE_MM2S': '0',
   'C_INCLUDE_MM2S_SF': '1',
   'C_MM2S_

In [4]:
IP_BASE_ADDRESS = 0x41210000
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

from pynq import MMIO
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

#data = 0xdeadbeef
#mmio.write(ADDRESS_OFFSET, data)
result = mmio.read(ADDRESS_OFFSET)
print(hex(result))

0x0


In [5]:
gps_chip = max2769_driver.MAX2769()


GPS chip is configured for IF=0MHz @ 4.096MSPS


In [6]:
gps_recorder = iotSDR_GPS_streamer.GPS_RX_Streaming(wordsPerBlk,overlay=overlay)

gps_recorder.start_fetching(recording_file,recording_time)

Recording started
Recording stoped 379
File writing Complete 379
streaming Stop


In [7]:
bitsPerSample = 4
samplingRate  = 4096000#*4
bitsPerblk    = gps_recorder.blk_count*32

totalBytes    = (samplingRate*recording_time*bitsPerSample)/8

with open(recording_file,"br") as f:
    data = f.read()
    #print(data)
    
print("Approx file size  :",totalBytes/1000000, "MB")
print("Actual file size  :",len(data)/1000000, "MB")

Approx file size  : 6.144 MB
Actual file size  : 6.209536 MB
